# MODNet on experimental dataset 

MODNet model with the GA, and doing joint learning on the experimental and PBE band gaps, and their difference.

In [1]:
def setup_threading():
    import os
    os.environ['OPENBLAS_NUM_THREADS'] = '1'
    os.environ['MKL_NUM_THREADS'] = '1'
    os.environ["OMP_NUM_THREADS"] = "1"
    os.environ["TF_NUM_INTRAOP_THREADS"] = "1"
    os.environ["TF_NUM_INTEROP_THREADS"] = "1"
    os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [2]:
setup_threading()

In [3]:
from modnet.preprocessing import MODData
from modnet.models.vanilla import MODNetModel
from modnet.hyper_opt.fit_genetic import FitGenetic
import numpy as np
import os
import copy

from collections import defaultdict
import itertools
import os
import pandas as pd
import matplotlib.pyplot as plt 
from IPython.display import Markdown
from matminer.datasets import load_dataset, get_all_dataset_info
from pymatgen.core import Composition

from modnet.featurizers import MODFeaturizer
from modnet.featurizers.presets import DeBreuck2020Featurizer

os.environ["CUDA_VISIBLE_DEVICES"] = "1"


If you use the ChemEnv tool for your research, please consider citing the following reference(s) :
David Waroquiers, Xavier Gonze, Gian-Marco Rignanese, Cathrin Welker-Nieuwoudt, Frank Rosowski,
Michael Goebel, Stephan Schenk, Peter Degelmann, Rute Andre, Robert Glaum, and Geoffroy Hautier,
"Statistical analysis of coordination environments in oxides",
Chem. Mater., 2017, 29 (19), pp 8346-8360,
DOI: 10.1021/acs.chemmater.7b02766



In [4]:
from sklearn.model_selection import KFold
from modnet.preprocessing import MODData

def shuffle_MD(data,random_state=10):
    data = copy.deepcopy(data)
    ids = data.df_targets.sample(frac=1,random_state=random_state).index
    data.df_featurized = data.df_featurized.loc[ids]
    data.df_targets = data.df_targets.loc[ids]
    data.df_structure = data.df_structure.loc[ids]
    
    return data

def MDKsplit(data,n_splits=5,random_state=10):
    data = shuffle_MD(data,random_state=random_state)
    ids = np.array(data.structure_ids)
    kf = KFold(n_splits=n_splits,shuffle=True,random_state=random_state)
    folds = []
    for train_idx, val_idx in kf.split(ids):
        data_train = MODData(data.df_structure.iloc[train_idx]['structure'].values,data.df_targets.iloc[train_idx].values,target_names=data.df_targets.columns,structure_ids=ids[train_idx])
        data_train.df_featurized = data.df_featurized.iloc[train_idx]
        #data_train.optimal_features = data.optimal_features
        
        data_val = MODData(data.df_structure.iloc[val_idx]['structure'].values,data.df_targets.iloc[val_idx].values,target_names=data.df_targets.columns,structure_ids=ids[val_idx])
        data_val.df_featurized = data.df_featurized.iloc[val_idx]
        #data_val.optimal_features = data.optimal_features

        folds.append((data_train,data_val))
        
    return folds

In [5]:
md = MODData.load('exp_gap_all')
df = pd.read_pickle('df_exp_cleaned.pkl')
md_joint = MODData.load('exp_pbe_joint')

2021-06-14 17:44:07,236 - modnet - INFO - Loaded <modnet.preprocessing.MODData object at 0x7f3e1ffb6d30> object, created with modnet version <=0.1.7
2021-06-14 17:44:07,865 - modnet - INFO - Loaded <modnet.preprocessing.MODData object at 0x7f3e94e40430> object, created with modnet version <=0.1.7


In [6]:
md_joint.df_targets

,exp_gap,pbe_gap,difference
mp-12699,0.00,0.0000,0.0000
mp-559459,3.40,2.1704,1.2296
mp-21162,0.00,0.0000,0.0000
mp-1306,0.00,0.0000,0.0000
mp-15252,0.53,0.1049,0.4251
...,...,...,...
mp-556541,1.80,1.5135,0.2865
mp-2602,1.04,0.0000,1.0400
mp-30366,0.00,0.0000,0.0000
mp-3718,1.45,0.2418,1.2082


In [7]:
md_joint.df_structure

,structure
id,
mp-12699,"[[ 0.13616363 -2.60631371 1.50340165] Ca, [-1..."
mp-559459,"[[2.14049282 6.43464803 1.76199846] Cs, [-0.70..."
mp-21162,"[[0. 0. 0.] Sr, [0. 2.5259875 2.5259875..."
mp-1306,"[[3.259129 1.88166243 1.15971025] Th, [ 3.25..."
mp-15252,"[[0.95531657 1.042239 0.86229681] Cu, [2.803..."
...,...
mp-556541,"[[0. 6.49363948 5.82657217] In, [ 0. ..."
mp-2602,"[[3.023318 3.023318 3.023318] Nd, [0. 0. 0.] As]"
mp-30366,"[[4.51774558 1.37529211 4.32659015] Ca, [1.109..."


In [8]:
import uncertainty_toolbox as uct

k = 5
random_state = 202010
folds = MDKsplit(md_joint,n_splits=k,random_state=random_state)
maes = np.ones(5)
uncertainties = np.ones(5)
metricss = []
for i,f in enumerate(folds):
    train = f[0]
    test = f[1]
    #train.feature_selection(n=-1, use_precomputed_cross_nmi=True)
    fpath = 'train_{}_{}'.format(random_state,i+1)
    if os.path.exists(fpath):
        train = MODData.load(fpath)
    else:
        train.feature_selection(n=-1, use_precomputed_cross_nmi=True)
        train.save(fpath)
      
    # assure no overlap
    assert len(set(train.df_targets.index).intersection(set(test.df_targets.index))) == 0
    
    ga = FitGenetic(train)
    model = ga.run(size_pop=20, num_generations=10, n_jobs=10)
    
    pred, std = model.predict(test, return_unc=True)
    true = test.df_targets
    metrics = uct.metrics.get_all_metrics(pred['exp_gap'].values, std['exp_gap'].values, true['exp_gap'].values)
    error = pred['exp_gap']-true['exp_gap']
    error = error.drop(pred.index[((pred['exp_gap']).abs()>20)]) # drop unrealistic values: happens extremely rarely
    std = std['exp_gap'].drop(std.index[((std['exp_gap']).abs()>20)]) # drop unrealistic values: happens extremely rarely
    mae = np.abs(error.values).mean()
    uncertainty = np.abs(std).mean()
    print('MAE =')
    print(mae)
    print('uncertainty =')
    print(uncertainty)
    maes[i] = mae
    uncertainties[i] = uncertainty
    metricss.append(metrics)

2021-06-14 17:44:08,036 - modnet - INFO - Loaded DeBreuck2020Featurizer featurizer.
2021-06-14 17:44:08,059 - modnet - INFO - Loaded DeBreuck2020Featurizer featurizer.
2021-06-14 17:44:08,069 - modnet - INFO - Loaded DeBreuck2020Featurizer featurizer.
2021-06-14 17:44:08,085 - modnet - INFO - Loaded DeBreuck2020Featurizer featurizer.
2021-06-14 17:44:08,095 - modnet - INFO - Loaded DeBreuck2020Featurizer featurizer.
2021-06-14 17:44:08,111 - modnet - INFO - Loaded DeBreuck2020Featurizer featurizer.
2021-06-14 17:44:08,121 - modnet - INFO - Loaded DeBreuck2020Featurizer featurizer.
2021-06-14 17:44:08,147 - modnet - INFO - Loaded DeBreuck2020Featurizer featurizer.
2021-06-14 17:44:08,160 - modnet - INFO - Loaded DeBreuck2020Featurizer featurizer.
2021-06-14 17:44:08,179 - modnet - INFO - Loaded DeBreuck2020Featurizer featurizer.
2021-06-14 17:44:09,053 - modnet - INFO - Loaded <modnet.preprocessing.MODData object at 0x7f3e0db3edc0> object, created with modnet version 0.1.9
2021-06-14 17

100%|██████████| 100/100 [14:08<00:00,  8.49s/it]

2021-06-14 17:58:19,170 - modnet - INFO - Loss per individual: ind 0: 0.600 	ind 1: 0.634 	ind 2: 0.619 	ind 3: 0.687 	ind 4: 0.655 	ind 5: 0.721 	ind 6: 0.597 	ind 7: 0.598 	ind 8: 0.668 	ind 9: 0.647 	ind 10: 0.651 	ind 11: 0.592 	ind 12: 0.635 	ind 13: 0.632 	ind 14: 0.758 	ind 15: 0.668 	ind 16: 0.623 	ind 17: 0.670 	ind 18: 0.639 	ind 19: 0.658 	


2021-06-14 17:58:20,876 - modnet - INFO - Generation number 1
2021-06-14 17:58:23,789 - modnet - INFO - Multiprocessing on 10 cores. Total of 128 cores available.


100%|██████████| 100/100 [12:48<00:00,  7.68s/it]

...

 (3/n) Calculating adversarial group calibration metrics
  [1/2] for mean absolute calibration error
Measuring adversarial group calibration by spanning group size between 0.0 and 1.0, in 10 intervals


 20%|██        | 2/10 [00:00<00:00, 10.71it/s]

  [2/2] for root mean squared calibration error
Measuring adversarial group calibration by spanning group size between 0.0 and 1.0, in 10 intervals


100%|██████████| 10/10 [00:00<00:00, 10.21it/s]

 (4/n) Calculating sharpness metrics
 (n/n) Calculating proper scoring rule metrics
**Finished Calculating All Metrics**


===================== Accuracy Metrics =====================
  MAE           0.395
  RMSE          0.764
  MDAE          0.034
  MARPD         137.516
  R2            0.807
  Correlation   0.901
=============== Average Calibration Metrics ================
  Root-mean-squared Calibration Error   0.158
  Mean-absolute Calibration Error       0.144
  Miscalibration Area                   0.145
========== Adversarial Group Calibration Metrics ===========
  Mean-absolute Adversarial Group Calibration Error
     Group Size: 0.11 -- Calibration Error: 0.197
     Group Size: 0.56 -- Calibration Error: 0.163
     Group Size: 1.00 -- Calibration Error: 0.144
  Root-mean-squared Adversarial Group Calibration Error
     Group Size: 0.11 -- Calibration Error: 0.225
     Group Size: 0.56 -- Calibration Error: 0.181
     Group Size: 1.00 -- Calibration Error: 0.158
==============

In [9]:
maes

array([0.36675306, 0.36639883, 0.28284831, 0.34679003, 0.39507955])

In [10]:
maes.mean()

0.3515739550393092

In [11]:
uncertainties

array([0.24604288, 0.23338228, 0.18500507, 0.19868283, 0.19745734])

In [12]:
uncertainties.mean()

0.21211408078670502

In [13]:
metricss

[{'accuracy': {'mae': 0.3667530573177938,
   'rmse': 0.7697104567030121,
   'mdae': 0.06484133243560786,
   'marpd': 133.59006994269183,
   'r2': 0.770480081543925,
   'corr': 0.8843674043278756},
  'avg_calibration': {'rms_cal': 0.12101189329577201,
   'ma_cal': 0.10626588465298142,
   'miscal_area': 0.10732491537610164},
  'adv_group_calibration': {'ma_adv_group_cal': {'group_sizes': array([0.        , 0.11111111, 0.22222222, 0.33333333, 0.44444444,
           0.55555556, 0.66666667, 0.77777778, 0.88888889, 1.        ]),
    'adv_group_cali_mean': array([0.41220707, 0.16939394, 0.14652472, 0.13480728, 0.12572608,
           0.12490139, 0.11849757, 0.11385443, 0.1120381 , 0.10626588]),
    'adv_group_cali_stderr': array([8.07998419e-02, 1.75891600e-02, 1.52666803e-02, 1.23398990e-02,
           1.06878253e-02, 8.77957956e-03, 5.01146067e-03, 4.42735620e-03,
           2.09615779e-03, 1.46284728e-17])},
   'rms_adv_group_cal': {'group_sizes': array([0.        , 0.11111111, 0.22222222, 

#### Conclusion

Relative percentage change of the error compared to MODNet_exp_GA reference: -2%